<a href="https://colab.research.google.com/github/Jacob-Rose-BU/Alternative-Investments---Assette-Capstone-Project/blob/main/PORTFOLIOGENERALINFORMATION_Table_Synthetic_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ALTERNATIVES_PORTFOLIOS Table – Synthetic Data Documentation
This synthetic dataset models metadata for alternative investment portfolios. It is intended to simulate real-world financial data for development, testing, and reporting purposes (e.g., factsheets, internal dashboards). The file is structured for Snowflake ingestion and integration into a broader data warehouse ecosystem.

| Column Name                | Description                                                                                      |
| -------------------------- | ------------------------------------------------------------------------------------------------ |
| `PORTFOLIOCODE`            | Unique code for each portfolio. Serves as the **primary key** and may be reused in other tables. |
| `NAME`                     | Name of the portfolio, including the strategy label (e.g., "Acme Capital Growth Strategy").      |
| `INVESTMENTSTYLE`          | Investment philosophy or style (e.g., Growth, Absolute Return).                                  |
| `PORTFOLIOCATEGORY`        | Broad classification of the portfolio (e.g., Alternatives, Fixed Income, Private Markets).       |
| `OPENDATE`                 | Date the portfolio was opened. Precedes inception and termination dates.                         |
| `PERFORMANCEINCEPTIONDATE` | Date when performance tracking began. Useful for YTD or benchmark comparison logic.              |
| `TERMINATIONDATE`          | Termination date of the portfolio (if applicable).                                               |
| `BASECURRENCYCODE`         | ISO 3-letter currency code (e.g., USD, EUR).                                                     |
| `BASECURRENCYNAME`         | Full currency name for clarity (e.g., US Dollar, Euro).                                          |
| `ISBEGINOFDAYPERFORMANCE`  | Flag indicating whether performance is tracked from the beginning of each trading day.           |
| `PRODUCTCODE`              | Foreign key reference to the related product or fund (from the PRODUCTMASTER table).             |

All dates are in YYYY-MM-DD format and follow a logical progression:
OPENDATE < PERFORMANCEINCEPTIONDATE < TERMINATIONDATE

Currencies and investment styles are drawn from standardized YAML configuration files to ensure modularity and consistency.

Keys like PORTFOLIOCODE and PRODUCTCODE may be reused in related tables
(e.g., fund performance, holdings, commentary, attribution) as foreign key proxies.

In [ ]:
import yaml
import pandas as pd
import random
from faker import Faker
from datetime import datetime
from google.colab import files

In [ ]:
uploaded = files.upload()  # Upload portfolio_config.yaml

In [ ]:
# Load the YAML config
with open("portfolio_config.yaml", "r") as f:
    config = yaml.safe_load(f)['portfolio_config']

# Initialize Faker and seed
fake = Faker()
random.seed(42)

In [ ]:
# Extract values from YAML
num_rows = config['num_rows']
investment_styles = config['investment_styles']
portfolio_categories = config['portfolio_categories']
currency_map = config['currency_map']
is_bod_perf_options = config['is_bod_perf_options']

# Generate unique PRODUCTCODEs to simulate FK
product_codes = [f"PRD{i:04d}" for i in range(1, num_rows + 1)]

In [ ]:
# Build synthetic rows
synthetic_portfolios = []

for i in range(num_rows):
    portfolio_code = f"PORT{i:04d}"
    style = random.choice(investment_styles)
    name = f"{fake.company()} {style} Strategy"
    category = random.choice(portfolio_categories)

    currency_code = random.choice(list(currency_map.keys()))
    currency_name = currency_map[currency_code]
    is_bod_perf = random.choice(is_bod_perf_options)
    product_code = random.choice(product_codes)

    # Generate dates in logical order
    open_date = fake.date_between(start_date='-10y', end_date='-5y')
    perf_inception_date = fake.date_between(start_date=open_date, end_date='-2y')
    termination_date = fake.date_between(start_date=perf_inception_date, end_date='today')

    # Add row
    synthetic_portfolios.append({
        "PORTFOLIOCODE": portfolio_code,
        "NAME": name,
        "INVESTMENTSTYLE": style,
        "PORTFOLIOCATEGORY": category,
        "OPENDATE": open_date,
        "PERFORMANCEINCEPTIONDATE": perf_inception_date,
        "TERMINATIONDATE": termination_date,
        "BASECURRENCYCODE": currency_code,
        "BASECURRENCYNAME": currency_name,
        "ISBEGINOFDAYPERFORMANCE": is_bod_perf,
        "PRODUCTCODE": product_code
    })

In [ ]:
# Convert to DataFrame
df_portfolios = pd.DataFrame(synthetic_portfolios)

# Save output
df_portfolios.to_csv("portfoliogeneralinformation_table_synthetic_data.csv", index=False)
print("✅ Synthetic data generated and saved as portfoliogeneralinformation_table_synthetic_data.csv")

# Preview
df_portfolios.head()

##Potential Snowflake Implementation
This code is intended not only to demonstrate realistic data generation for academic or testing purposes but also to support end-to-end data pipelines where Snowflake acts as the target data warehouse. With minimal adjustments (e.g., schema renaming, batch control), this data can be used in:
* Report generation (e.g., Assette)
* Performance attribution systems
* ESG dashboards
* Fund analytics platforms

The code below shows a programmatic load using snowflake.connector in Python. Please see example.env to include your Snowflake database and API keys.

In [ ]:
import snowflake.connector

# Load credentials
load_dotenv()

sf_user = os.getenv("SNOWFLAKE_USER")
sf_password = os.getenv("SNOWFLAKE_PASSWORD")
sf_account = os.getenv("SNOWFLAKE_ACCOUNT")
sf_database = os.getenv("SNOWFLAKE_DATABASE")
sf_schema = os.getenv("SNOWFLAKE_SCHEMA")
sf_warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")

# Connect to Snowflake
conn = snowflake.connector.connect(
    user=sf_user,
    password=sf_password,
    account=sf_account,
    warehouse=sf_warehouse,
    database=sf_database,
    schema=sf_schema
)

cursor = conn.cursor()

# Function to upload DataFrame
def append_to_snowflake(df, table_name):
    try:
        # Create temp CSV
        temp_csv = "/tmp/temp_fund_upload.csv"
        df.to_csv(temp_csv, index=False)

        # Create staging area in memory
        cursor.execute(f"PUT file://{temp_csv} @%{table_name} OVERWRITE = TRUE")

        # Copy from staged CSV to table
        columns = ",".join(df.columns)
        cursor.execute(f"""
            COPY INTO {table_name}
            FROM @%{table_name}
            FILE_FORMAT = (TYPE = CSV FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER=1)
        """)

        print(f"✅ Data appended to {table_name} in Snowflake")

    except Exception as e:
        print("❌ Failed to upload data:", e)
    finally:
        cursor.close()
        conn.close()

# Example usage:
append_to_snowflake(df_portfolios, "PORTFOLIOGENERALINFORMATION")